### Vorbereitung ✅

*Ziel:*
- Jeder Stadt wird ein repräsentativer Verkehrskennwert zugeordnet.
- Die endgültige Kennzahl wird pro Stadt und pro Kopf berechnet.

In [2]:
import pandas as pd
import numpy as np

parquet_path = './Datasets/Parquet Files'

In [3]:
df_zaehlstellen_verkehr = pd.read_parquet(f"{parquet_path}/roh_selected_deutsch.parquet")
df_zaehlstellen_meta = pd.read_parquet(f"{parquet_path}/zaehlstellen_top30.parquet")
df_zaehlstellen_meta = df_zaehlstellen_meta.rename(columns={'GEN': 'Stadt'})

### Merge von den beiden Datasets ✅

In [4]:
import pandas as pd

roh = pd.read_parquet('./Datasets/Parquet Files/roh_selected_deutsch.parquet')

# Merge-Schlüssel in str umwandeln (nur für die Schlüsselspalten)
df_zaehlstellen_meta['Dauerzaehlstellennummer'] = df_zaehlstellen_meta['Dauerzaehlstellennummer'].astype(str)
roh['Zählstellennummer'] = roh['Zählstellennummer'].astype(str)

# Merge durchführen
merged = df_zaehlstellen_meta.merge(
    roh,
    left_on='Dauerzaehlstellennummer',
    right_on='Zählstellennummer',
    how='left',
    suffixes=("", "_rechts")
)

# Spalten definieren, die behalten werden
columns_to_keep = ["Dauerzaehlstellennummer",
                   "Straßenklasse", 
                   "Datum", 
                   "Stunde", 
                   "Stadt", 
                   "Verkehrsmenge alle Kfz Richtung 1",
                   "Verkehrsmenge alle Kfz Richtung 2"]  # <- Hier deine Spaltennamen eintragen

# DataFrame auf die gewünschten Spalten reduzieren
merged = merged[columns_to_keep]

# Die gewünschten Summenspalten definieren
sum_cols = [
    "Verkehrsmenge alle Kfz Richtung 1",
    "Verkehrsmenge alle Kfz Richtung 2"
]

# Numerische Konvertierung der Verkehrs-Spalten
merged[sum_cols] = merged[sum_cols].apply(pd.to_numeric, errors='coerce')


merged.head()

,Dauerzaehlstellennummer,Straßenklasse,Datum,Stunde,Stadt,Verkehrsmenge alle Kfz Richtung 1,Verkehrsmenge alle Kfz Richtung 2
0,1194,A,240101,1,Kiel,163,324
1,1194,A,240101,2,Kiel,495,543
2,1194,A,240101,3,Kiel,487,424
3,1194,A,240101,4,Kiel,307,258
4,1194,A,240101,5,Kiel,158,142


### Entfernen von Duplikaten ✅

In den Daten sind 288 Zeilen pro Tag enthalten. Es gibt zu jedem Tag 12 mal die gleiche Abfolge an Datenzeilen (12 mal 24 Stunden pro Datum = 288 Datenzeilen pro Datum). Im nächsten Schritt gehen wir von 288 auf 24 pro Datum, indem wir mit `drop_duplicates` nur die erste "Charge" (`keep="first"`) behalten.

In [5]:
# Annahme: Spalte namens "Stunde" geht von 1 bis 24

filtered_merged = (
    merged
    .sort_values(["Dauerzaehlstellennummer", "Stadt", "Datum", "Stunde", "Straßenklasse"])
    .drop_duplicates(subset=["Dauerzaehlstellennummer", "Stadt", "Datum", "Stunde", "Straßenklasse"], keep="first")
    .reset_index(drop=True)
)

# Ergebnis anzeigen (optional)
filtered_merged.head()


,Dauerzaehlstellennummer,Straßenklasse,Datum,Stunde,Stadt,Verkehrsmenge alle Kfz Richtung 1,Verkehrsmenge alle Kfz Richtung 2
0,1111,B,240101,1,Kiel,127,117
1,1111,B,240101,10,Kiel,403,523
2,1111,B,240101,11,Kiel,503,610
3,1111,B,240101,12,Kiel,602,603
4,1111,B,240101,13,Kiel,639,627


In [6]:
# Schritt 1: Zeilenanzahl pro Stadt und Datum VOR dem Entfernen der Duplikate
before_counts = (
    merged.groupby(['Dauerzaehlstellennummer', 'Stadt', 'Datum'])
    .size()
    .reset_index(name='Anzahl_vorher')
)

# Schritt 2: Zeilenanzahl pro Stadt und Datum NACH dem Entfernen der Duplikate
after_counts = (
    filtered_merged.groupby(['Dauerzaehlstellennummer', 'Stadt', 'Datum'])
    .size()
    .reset_index(name='Anzahl_nachher')
)

# Schritt 3: Zusammenführen und Differenz berechnen
dupe_stats = before_counts.merge(after_counts, on=['Dauerzaehlstellennummer', 'Stadt', 'Datum'], how='left')
dupe_stats['Entfernte_Duplikate'] = dupe_stats['Anzahl_vorher'] - dupe_stats['Anzahl_nachher']

# Optional: nur Städte/Datums mit entfernten Duplikaten anzeigen
dupe_stats = dupe_stats[dupe_stats['Entfernte_Duplikate'] > 0]

In [7]:
dupe_stats.head()

,Dauerzaehlstellennummer,Stadt,Datum,Anzahl_vorher,Anzahl_nachher,Entfernte_Duplikate
0,1111,Kiel,240101,288,24,264
1,1111,Kiel,240102,288,24,264
2,1111,Kiel,240103,288,24,264
3,1111,Kiel,240104,288,24,264
4,1111,Kiel,240105,288,24,264


In [8]:
# Wie oft kommt jede Anzahl_vorher in dupe_stats vor?
Anzahl_vorher = dupe_stats['Anzahl_vorher'].value_counts().sort_index()
Anzahl_nachher = dupe_stats['Anzahl_nachher'].value_counts().sort_index()

print("Anzahl_vorher:")
for wert, anzahl in Anzahl_vorher.items():
    print(f"  In {anzahl} Fällen gab es jeweils {wert} Zeilen vor dem Entfernen der Duplikate (also {anzahl} Kombinationen aus Dauerzaehlstellennummer, Stadt und Datum mit jeweils {wert} Zeilen).")


print("\nAnzahl_nachher:")
for wert, anzahl in Anzahl_nachher.items():
    print(f"  Nach dem Entfernen der Duplikate gibt es noch {wert} Zeilen für alle ({anzahl}) Kombinationen aus Dauerzaehlstellennummer, Stadt und Datum.")

Anzahl_vorher:
  In 77439 Fällen gab es jeweils 288 Zeilen vor dem Entfernen der Duplikate (also 77439 Kombinationen aus Dauerzaehlstellennummer, Stadt und Datum mit jeweils 288 Zeilen).

Anzahl_nachher:
  Nach dem Entfernen der Duplikate gibt es noch 24 Zeilen für alle (77439) Kombinationen aus Dauerzaehlstellennummer, Stadt und Datum.


### Berechnung von Tageswerten ✅

Zunächst werden in `tageswerte` die Verkehrsmengen mit `.sum()` pro `'Dauerzaehlstellennummer', 'Stadt', 'Datum', 'Straßenklasse'` addiert.

In [9]:
tageswerte = (
    filtered_merged.groupby(['Dauerzaehlstellennummer', 'Stadt', 'Datum', 'Straßenklasse'])[sum_cols]
    .sum()
    .reset_index()
)
tageswerte.head()

,Dauerzaehlstellennummer,Stadt,Datum,Straßenklasse,Verkehrsmenge alle Kfz Richtung 1,Verkehrsmenge alle Kfz Richtung 2
0,1111,Kiel,240101,B,7686,9058
1,1111,Kiel,240102,B,15690,17259
2,1111,Kiel,240103,B,16777,17249
3,1111,Kiel,240104,B,18293,18384
4,1111,Kiel,240105,B,16501,17477


Nun werden `Kfz_gesamt` berechnet, indem ``Verkehrsmenge alle Kfz Richtung 1`` und ``Verkehrsmenge alle Kfz Richtung 2`` addiert werden.

In [10]:
# Gruppierung: Anzahl der Zeilen (z.B. Stunden) berechnen
anzahl_stunden = (
    filtered_merged.groupby(['Dauerzaehlstellennummer', 'Stadt', 'Datum'])
    .size()
    .reset_index(name="Anzahl_Stunden")
)

# Beide Ergebnisse mergen
tageswerte = tageswerte.merge(
    anzahl_stunden, 
    on=['Dauerzaehlstellennummer', 'Stadt', 'Datum'], 
    how='left',
    suffixes=("", "_rechts")
)

# Gesamtes Verkehrsaufkommen berechnen
tageswerte["Kfz_gesamt"] = (
    tageswerte["Verkehrsmenge alle Kfz Richtung 1"] +
    tageswerte["Verkehrsmenge alle Kfz Richtung 2"]
)


# Die gewünschten Summenspalten definieren
sum_cols2 = [
    "Verkehrsmenge alle Kfz Richtung 1",
    "Verkehrsmenge alle Kfz Richtung 2",
    "Kfz_gesamt"
]

# Numerische Konvertierung der Verkehrs-Spalten
tageswerte[sum_cols2] = tageswerte[sum_cols2].apply(pd.to_numeric, errors='coerce')

# Ergebnis anzeigen
tageswerte.head()

,Dauerzaehlstellennummer,Stadt,Datum,Straßenklasse,Verkehrsmenge alle Kfz Richtung 1,Verkehrsmenge alle Kfz Richtung 2,Anzahl_Stunden,Kfz_gesamt
0,1111,Kiel,240101,B,7686,9058,24,16744
1,1111,Kiel,240102,B,15690,17259,24,32949
2,1111,Kiel,240103,B,16777,17249,24,34026
3,1111,Kiel,240104,B,18293,18384,24,36677
4,1111,Kiel,240105,B,16501,17477,24,33978


In [12]:
# 8. Zweitens: Mittelwert der Tageswerte je Zählstelle (→ typischer Tag)
zaehlstelle_tagesmittel = (
    tageswerte.groupby(['Dauerzaehlstellennummer', 'Stadt', 'Straßenklasse'])[sum_cols2]
    .mean()
    .reset_index()
)

zaehlstelle_tagesmittel = zaehlstelle_tagesmittel.rename(columns={
    "Kfz_gesamt": "Kfz_Mean_Pro_Tag", 
    "Verkehrsmenge alle Kfz Richtung 1": "Kfz_mean_pro_Tag_Richtung_1", 
    "Verkehrsmenge alle Kfz Richtung 2": "Kfz_mean_pro_Tag_Richtung_2"
    })

zaehlstelle_tagesmittel.head()

,Dauerzaehlstellennummer,Stadt,Straßenklasse,Kfz_mean_pro_Tag_Richtung_1,Kfz_mean_pro_Tag_Richtung_2,Kfz_Mean_Pro_Tag
0,1111,Kiel,B,7786.610329,7496.774648,15283.384977
1,1112,Kiel,B,9698.629108,9651.032864,19349.661972
2,1158,Kiel,B,9225.797386,8207.202614,17433.000000
3,1194,Kiel,A,34522.620219,36165.420765,70688.040984
4,2002,Berlin,A,39207.021858,40595.647541,79802.669399


### Gewichtung der Straßen ✅

In den ursprünglichen Datasets gibt es die Straßenklassen A und B.
- Straßenklasse A = Autobahnen (Bundesautobahnen, BAB)
- Straßenklasse B = Bundesstraßen (klassische Überlandstraßen, B‑Straßen) 

In [13]:
# 1. Definiere deine Gewichtungsfaktoren
gewichte = {'A': 0.8, 'B': 1.2}

# 2. Füge eine Spalte für das Gewicht hinzu
zaehlstelle_tagesmittel['Gewicht'] = zaehlstelle_tagesmittel['Straßenklasse'].map(gewichte)

# 3. Berechne die gewichtete mittlere Tagesfahrleistung
zaehlstelle_tagesmittel['Kfz_Mean_Pro_Tag_gew'] = (
    zaehlstelle_tagesmittel['Kfz_Mean_Pro_Tag'] * zaehlstelle_tagesmittel['Gewicht']
)

# Optional: Ergebnis anschauen
zaehlstelle_tagesmittel[['Dauerzaehlstellennummer', 'Stadt', 'Straßenklasse',
                         'Kfz_Mean_Pro_Tag', 'Gewicht', 'Kfz_Mean_Pro_Tag_gew']].head(15)


,Dauerzaehlstellennummer,Stadt,Straßenklasse,Kfz_Mean_Pro_Tag,Gewicht,Kfz_Mean_Pro_Tag_gew
0,1111,Kiel,B,15283.384977,1.2,18340.061972
1,1112,Kiel,B,19349.661972,1.2,23219.594366
2,1158,Kiel,B,17433.000000,1.2,20919.600000
3,1194,Kiel,A,70688.040984,0.8,56550.432787
4,2002,Berlin,A,79802.669399,0.8,63842.135519
5,2003,Berlin,A,141329.303279,0.8,113063.442623
6,2004,Berlin,A,4321.978142,0.8,3457.582514
7,2005,Berlin,A,117416.825137,0.8,93933.460109
8,2006,Berlin,A,147278.377049,0.8,117822.701639
9,2008,Berlin,A,98409.923497,0.8,78727.938798


In [45]:
zaehlstelle_tagesmittel.to_parquet(f"{parquet_path}/zaehlstelle_tagesmittel.parquet")

In [65]:
sum_cols3 = [
    "Kfz_mean_pro_Tag_Richtung_1",
    "Kfz_mean_pro_Tag_Richtung_2",
    "Kfz_Mean_Pro_Tag",
    "Kfz_Mean_Pro_Tag_gew"
]

# Zählstellen mit 0 Verkehrsaufkommen in beiden Richtungen entfernen
zaehlstelle_tagesmittel_clean = zaehlstelle_tagesmittel[
    ~(
        (zaehlstelle_tagesmittel["Kfz_mean_pro_Tag_Richtung_1"] == 0) &
        (zaehlstelle_tagesmittel["Kfz_mean_pro_Tag_Richtung_2"] == 0)
    )
].copy()

# Mittelwerte über Städte berechnen (wie bisher)
stadt_agg = (
    zaehlstelle_tagesmittel_clean.groupby('Stadt')[sum_cols3]
    .mean()
    .reset_index()
)

# Anzahl der Zählstellen pro Stadt, die nicht nur Nullen haben
anzahl_zaehlstellen = (
    zaehlstelle_tagesmittel_clean.groupby('Stadt')["Dauerzaehlstellennummer"]
    .nunique()
    .reset_index(name="Anzahl_Zaehlstellen")
)

# Mergen
stadt_agg = stadt_agg.merge(anzahl_zaehlstellen, on="Stadt", how="left")

# Ergebnis anzeigen
stadt_agg.head()


,Stadt,Kfz_mean_pro_Tag_Richtung_1,Kfz_mean_pro_Tag_Richtung_2,Kfz_Mean_Pro_Tag,Kfz_Mean_Pro_Tag_gew,Anzahl_Zaehlstellen
0,Aachen,14202.046175,13835.253552,28037.299727,24883.090601,10
1,Augsburg,49700.505464,49646.161202,99346.666667,79477.333333,1
2,Berlin,32892.774831,33762.315493,66655.090325,56405.857666,17
3,Bielefeld,35386.989071,34512.035974,69899.025046,63180.500182,6
4,Bochum,45796.399818,46296.747723,92093.147541,73674.518033,3


### Runden der Werte 🤷‍♀️

Runden der Werte in `stadt_agg` und `zaehlstelle_tagesmittel`

In [66]:
stadt_agg_rounded = stadt_agg.copy()
cols_to_round = sum_cols3

for col in cols_to_round:
    stadt_agg_rounded[col] = stadt_agg_rounded[col].round(0).astype(int)

zaehlstelle_rounded = zaehlstelle_tagesmittel.copy()
for col in cols_to_round:
    zaehlstelle_rounded[col] = zaehlstelle_rounded[col].round(0).astype(int)


In [67]:
stadt_agg_rounded.to_parquet(f"{parquet_path}/Verkehrsmenge_pro_Stadt.parquet")
zaehlstelle_rounded.to_parquet(f"{parquet_path}/Verkehrsmittel_pro_Zaehlstelle.parquet")

In [68]:
stadt_agg_rounded.head()

,Stadt,Kfz_mean_pro_Tag_Richtung_1,Kfz_mean_pro_Tag_Richtung_2,Kfz_Mean_Pro_Tag,Kfz_Mean_Pro_Tag_gew,Anzahl_Zaehlstellen
0,Aachen,14202,13835,28037,24883,10
1,Augsburg,49701,49646,99347,79477,1
2,Berlin,32893,33762,66655,56406,17
3,Bielefeld,35387,34512,69899,63181,6
4,Bochum,45796,46297,92093,73675,3


## Berechnung Verkehr Pro Kopf ✅

### Einmalige Erstellung von DF mit EinwohnerZahl pro Stadt ✅

In [34]:
import pandas as pd
import sqlite3

# 1. Parquet-Datei mit den Top-30-Gemeinden einlesen
df_geo = pd.read_parquet('./Datasets/Parquet Files/gemeinden_top30_geo.parquet')
# Achte darauf, dass die Spalten "GEN", "AGS", "ARS", "SDV_ARS" vorhanden sind

# 2. Verbindung zur regionalen DB aufbauen
conn = sqlite3.connect('regionale_datenbank.db')

# 3. Regionale Tabelle laden: Ort, Einwohnerspanne, Amtlicher Regionalschlüssel
query = """
SELECT Ort AS GEN, 
       'Amtlicher Regionalschlüssel' AS Regionalschluessel, 
       Einwohnerzahl
FROM Bevoelkerung
"""
df_reg = pd.read_sql_query(query, conn)

conn.close()

# 4. Join auf GEN (Gemeindename)
df_stadt = (
    df_geo
    .merge(df_reg, on='GEN', how='left', suffixes=('', '_db'))
)

# 5. Gewünschte Spalten ausgeben
columns = [
    'Regionalschluessel', 'Einwohnerzahl',
    'GEN', 'AGS', 'ARS', 'SDV_ARS'
]
df_stadt = df_stadt[columns]

# Ausgabe prüfen
df_stadt.head()


,Regionalschluessel,Einwohnerzahl,GEN,AGS,ARS,SDV_ARS
0,Amtlicher Regionalschlüssel,577572,Aachen,05334002,053340002002,053340002002
1,Amtlicher Regionalschlüssel,261178,Aachen,05334002,053340002002,053340002002
2,Amtlicher Regionalschlüssel,1893005,Augsburg,09761000,097610000000,097610000000
3,Amtlicher Regionalschlüssel,294647,Augsburg,09761000,097610000000,097610000000
4,Amtlicher Regionalschlüssel,294647,Augsburg,09761000,097610000000,097610000000


An dieser Stelle sind für manche Städte mehrere Einträge vorhanden. Wir wollen den Eintrag auswählen, der die geringste Einwohnerzahl hat. Die anderen Einträge sind meistens Städteregionen oder Stadtkreise.

In [35]:
# 1) Für jede Stadt den Index der Zeile mit dem minimalen Einwohnwert finden
idx = df_stadt.groupby('GEN')['Einwohnerzahl'].idxmin()

# 2) Nur diese Zeilen extrahieren
df_stadt_core = df_stadt.loc[idx].reset_index(drop=True)
df_stadt_core = df_stadt_core.rename(columns={'GEN': 'Stadt'})


df_stadt_core.to_parquet(f"{parquet_path}/df_stadt_einwohner.parquet")

### Berechnung Verkehr pro Kopf auf Stadtbasis ✅

In [ ]:
df_verkehr = pd.read_parquet(f"{parquet_path}/Verkehrsmittel_pro_Zaehlstelle.parquet")
df_stadt = pd.read_parquet(f"{parquet_path}/df_stadt_einwohner.parquet")

In [70]:
import pandas as pd
import numpy as np

np.random.seed(1337)

cols_to_sum = [
    'Kfz_mean_pro_Tag_Richtung_1',
    'Kfz_mean_pro_Tag_Richtung_2',
    'Kfz_Mean_Pro_Tag',
    'Kfz_Mean_Pro_Tag_gew'
]

# --- 2) Aggregation pro Stadt + Bootstrap ---

def agg_pro_stadt(df_v, df_s, n_boot=1000):
    records = []
    for stadt, grp in df_v.groupby('Stadt'):
        n = len(grp)
        werte = grp['Kfz_Mean_Pro_Tag_gew'].dropna().values
        if n < 2 or werte.size < 2:
            continue
        #boot_means = [np.mean(np.random.choice(werte, size=len(werte), replace=True)) for _ in range(n_boot)]

        rng = np.random.default_rng(1337)
        boot_means = [np.mean(rng.choice(werte, size=len(werte), replace=True)) for _ in range(n_boot)]
          
        mean = np.mean(boot_means)
        se = np.std(boot_means, ddof=1)
        ci_lo, ci_hi = np.percentile(boot_means, [2.5, 97.5])
        einw = df_s.loc[df_s['Stadt'] == stadt, 'Einwohnerzahl'].values
        if einw.size == 0:
            continue
        ew = einw[0]
        records.append({
            'Stadt': stadt,
            'Anzahl_Zaehlen': n,
            'Kfz_Mean': mean,
            'Kfz_SE': se,
            'Kfz_CI_lo': ci_lo,
            'Kfz_CI_hi': ci_hi,
            'Einwohnerzahl': ew,
            'Verkehr_pro_Kopf': mean / ew
        })
    return pd.DataFrame(records)

stadt_agg = agg_pro_stadt(df_verkehr, df_stadt, n_boot=1000)

Kfz_Mean = Durchschnittliche Anzahl Fahrzeuge, die an einem typischen Tag pro Zählstelle in dieser Stadt gemessen werden

Verkehr_pro_Kopf = Durchschnittlicher Gesamtverkehr pro Kopf: `Kfz_Mean` geteilt durch `Einwohnerzahl`

Kfz_SE = Standardfehler dieses Mittels: Gibt an, wie stark die Werte zwischen den Zählstellen schwanken (je mehr Zählstellen, desto kleiner SE).

Kfz_CI_lo = Untere Grenze des 95 %-Konfidenzintervalls (aus dem Bootstrap).

Kfz_CI_hi =	Obere Grenze des 95 %-Konfidenzintervalls (aus dem Bootstrap).

In [76]:
# Alle Städte in df_verkehr und stadt_agg
alle_staedte = set(df_verkehr['Stadt'].unique())
agg_staedte = set(stadt_agg['Stadt'].unique())

# Fehlende Städte (in df_verkehr, aber nicht in stadt_agg)
fehlende = alle_staedte - agg_staedte
print("Fehlende Städte:", fehlende)


Fehlende Städte: {'Chemnitz', 'Augsburg'}


Die Städte Chemnitz und Augsburg haben nur 1 Zählstelle. Deswegen werden sie hier aus dem Modell fallengelassen, da sie letztendlich nicht genügend Messpunkte für eine Aggregation vorhanden. Mit nur einer Zählstelle lässt sich kein Konfidenzintervall oder Standardfehler berechnen, und die Aussagekraft ist extrem limitiert.

In [77]:
stadt_agg['Verkehr_pro_1000_EW'] = stadt_agg['Verkehr_pro_Kopf'] * 1000

# Optional: Ergebnis anschauen
stadt_agg.head(141)


,Stadt,Anzahl_Zaehlen,Kfz_Mean,Kfz_SE,Kfz_CI_lo,Kfz_CI_hi,Einwohnerzahl,Verkehr_pro_Kopf,Verkehr_pro_1000_EW
0,Aachen,12,21028.194500,4192.677424,13325.629167,29573.920833,261178,0.080513,80.512886
1,Berlin,17,56013.352941,8961.201550,39057.397059,74004.086765,3596999,0.015572,15.572246
2,Bielefeld,6,63071.907000,7748.799174,48039.570833,77535.333333,330072,0.191085,191.085300
3,Bochum,6,37485.657333,16244.499939,8568.333333,69283.150000,354288,0.105806,105.805608
4,Bonn,6,18369.404833,10429.968284,2776.833333,40802.500000,321544,0.057129,57.128744
5,Braunschweig,9,42685.037333,5725.868315,31279.822222,53258.836111,252816,0.168838,168.838354
6,Bremen,5,59734.472400,8342.609347,45411.600000,77869.600000,575071,0.103873,103.873213
7,Dortmund,8,58708.622500,5376.261571,48234.418750,69297.665625,598246,0.098135,98.134584
8,Dresden,2,60013.486000,12143.040609,43277.000000,78000.000000,557782,0.107593,107.593085
9,Duisburg,4,46388.974000,13604.948057,21086.750000,71492.000000,501415,0.092516,92.516127


In [78]:
stadt_agg.to_parquet(f"{parquet_path}/stadt_agg.parquet")